In [37]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TimeDistributed, Flatten
from tensorflow.keras.layers import LSTM,RepeatVector
from tensorflow.keras.layers import GlobalMaxPooling1D, RepeatVector,Conv1D


# 加载数据
data = pd.read_csv('daily_stock_data.csv',header=None)  # 确保文件路径是正确的
data.sort_index(ascending=False, inplace=True)
data = data.reset_index(drop=True)
data.columns = ['price', 'vol']


# 计算差值
data_diff = data.diff().fillna(0)

# 数据缩放
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_diff)

def generator(data, lookback=200, delay=10, batch_size=32):  # 将delay设置为100
    max_index = len(data) - delay - 1
    i = lookback
    while 1:
        if i + delay >= max_index:
            i = lookback
        samples = np.zeros((batch_size, lookback, data.shape[-1]))
        targets = np.zeros((batch_size, delay, data.shape[-1]))  # Change targets to be 'delay' length sequences
        for j in range(batch_size):
            if i + delay >= max_index:
                i = lookback
            rows = np.arange(i - lookback, i)
            samples[j] = data[rows]
            targets[j] = data[i: i + delay]  # Change targets to be 'delay' length sequences
            i += 1
        yield samples, targets

lookback = 200
delay = 30  # Set delay to 100 for predicting 100 steps ahead
train_gen = generator(data_scaled, lookback=lookback, delay=delay)

model_path = 'stock_model.h5'  # 确保路径是正确的

if os.path.exists(model_path):
    # 加载模型
    model = load_model(model_path)
else:
    # 定义模型
    inputs = Input(shape=(lookback, data.shape[-1]))
    x = MultiHeadAttention(num_heads=2, key_dim=2)(inputs, inputs)
    x = Dropout(0.5)(x)

    # 减小序列长度
    x = Conv1D(64, 10, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)

    # 扩展序列长度到100
    x = RepeatVector(delay)(x)
    x = LSTM(100, return_sequences=True)(x)  # LSTM 层，输出形状为 (batch_size, timesteps, units)

    outputs = TimeDistributed(Dense(data.shape[-1]))(x)  # 用TimeDistributed包装Dense层，以适应新的目标序列长度
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(), loss='mse')
    model.fit(train_gen, epochs=30, steps_per_epoch=10)
    
    # 保存模型
    model.save(model_path)



Epoch 1/30


2023-07-03 22:02:49.966704: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 2s 25ms/step - loss: 0.3941
Epoch 2/30
10/10 [==============================] - 0s 25ms/step - loss: 1.5745
Epoch 3/30
10/10 [==============================] - 0s 23ms/step - loss: 1.7770
Epoch 4/30
10/10 [==============================] - 0s 25ms/step - loss: 0.2915
Epoch 5/30
10/10 [==============================] - 0s 24ms/step - loss: 1.3779
Epoch 6/30
10/10 [==============================] - 0s 25ms/step - loss: 1.9071
Epoch 7/30
10/10 [==============================] - 0s 24ms/step - loss: 0.4982
Epoch 8/30
10/10 [==============================] - 0s 27ms/step - loss: 1.1006
Epoch 9/30
10/10 [==============================] - 0s 25ms/step - loss: 1.9777
Epoch 10/30
10/10 [==============================] - 0s 26ms/step - loss: 0.7893
Epoch 11/30
10/10 [==============================] - 0s 28ms/step - loss: 0.7729
Epoch 12/30
10/10 [==============================] - 0s 26ms/step - loss: 2.1097
Epoch 13/30
10/10 [=============================

/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [38]:
# model.summary()
data_to_predict = data_scaled[-lookback:]  # 最后lookback个数据点
datadiff_to_predict = data_diff[-lookback:]
print(datadiff_to_predict)
samples = np.reshape(data_to_predict, (1, lookback, data.shape[-1]))  # 调整samples的shape为(1, lookback, data.shape[-1])
predictions = model.predict(samples)  # 一次性预测出未来100个时间点的数据
predicted_data = predictions[0]  # 获得预测值
# 对最后100个预测结果进行反归一化
predicted_data_diff = scaler.inverse_transform(predicted_data)
print(predicted_data_diff)

# 使用原始数据的最后一个值作为初始值，然后使用预测的差值序列进行累加
last_value = data.values[-1, :]  # 使用原始数据的最后一个值
predicted_data1 = np.zeros_like(predicted_data_diff)
predicted_data1[0, :] = last_value + predicted_data_diff[0, :]  # 对第一步做特殊处理
for i in range(1, predicted_data_diff.shape[0]):
    predicted_data1[i, :] = predicted_data1[i - 1, :] + predicted_data_diff[i, :]

# 保存预测结果到csv文件
predicted_df = pd.DataFrame(predicted_data1, columns=['predicted_price', 'predicted_vol'])
predicted_df.to_csv('predicted_data_transformer.csv', index=False)


      price        vol
1074  -5.00   17028.32
1075  26.39  130707.28
1076   0.00  -77567.68
1077  -5.21   44108.02
1078  -8.59 -100841.47
...     ...        ...
1269  -2.37   11477.10
1270   1.36  -10650.32
1271   2.69   62457.58
1272   9.50   64964.60
1273  -1.18  -42259.61

[200 rows x 2 columns]
1/1 [==============================] - 0s 214ms/step
[[9.8768145e-02 1.2817190e+03]
 [1.6620907e-01 2.4337739e+03]
 [2.4572177e-01 3.2135728e+03]
 [3.0651689e-01 3.7187883e+03]
 [3.4682599e-01 4.0508274e+03]
 [3.7182605e-01 4.2799893e+03]
 [3.8669255e-01 4.4472451e+03]
 [3.9519399e-01 4.5748750e+03]
 [3.9980885e-01 4.6750400e+03]
 [4.0210101e-01 4.7547578e+03]
 [4.0303370e-01 4.8185273e+03]
 [4.0319911e-01 4.8695273e+03]
 [4.0295061e-01 4.9102124e+03]
 [4.0250126e-01 4.9425513e+03]
 [4.0197578e-01 4.9681660e+03]
 [4.0144405e-01 4.9883984e+03]
 [4.0094411e-01 5.0043271e+03]
 [4.0049368e-01 5.0168525e+03]
 [4.0009913e-01 5.0266836e+03]
 [3.9976108e-01 5.0343940e+03]
 [3.9947549e-01 5.0404438e+